# Analysis of unsupervised models

In [1]:
import os

import numpy as np
import pandas as pd

evaluation_dir = os.path.join(os.getcwd(), '..', '..', 'evaluation')
predictions_file = os.path.join(evaluation_dir, 'supervised.csv')

predictions = pd.read_csv(predictions_file)
predictions['signal'] = predictions['signal'].apply(eval).apply(np.array)
predictions['duration'] = pd.to_timedelta(predictions['duration'])

ground_truth_file = os.path.join(evaluation_dir, 'ground_truth.csv')
ground_truth = pd.read_csv(ground_truth_file)
ground_truth['signal'] = ground_truth['signal'].apply(eval).apply(np.array)

predictions.head()

,subject,setting,method,execution_time,sample_rate,signal
0,Proband01,101_natural_lighting,mtts_can,0 days 00:00:09.877512,30,"[0.5544167160987854, 0.9687221050262451, 1.162..."
1,Proband01,101_natural_lighting,big_small,0 days 00:03:24.014127,30,"[0.07677064090967178, 0.07124900072813034, -0...."
2,Proband02,101_natural_lighting,mtts_can,0 days 00:00:16.431936,30,"[0.2787117063999176, 0.13486619293689728, 0.48..."
3,Proband02,101_natural_lighting,big_small,0 days 00:02:37.844673,30,"[-0.5299593210220337, -0.5200148224830627, -0...."
4,Proband03,101_natural_lighting,mtts_can,0 days 00:00:23.953848,30,"[-0.07859578728675842, -0.4248553514480591, -0..."


## Frequency extraction evaluation

In [ ]:
import respiration.analysis as analysis
import tqdm.auto as tqdm

lowpass = 0.08
highpass = 0.6

analysis_results = []

for index, row in tqdm.tqdm(predictions.iterrows(), total=len(predictions)):
    subject, setting = row['subject'], row['setting']

    # Get the ground truth signal and sampling rate
    gt_signal, gt_sampling_rate = ground_truth[
        (ground_truth['subject'] == subject) &
        (ground_truth['setting'] == setting)
        ][['signal', 'sample_rate']].values[0]

    prediction, sampling_rate = row['signal'], row['sample_rate']

    compare = analysis.SignalCompare(
        prediction,
        sampling_rate,
        gt_signal,
        gt_sampling_rate,
    )

    for metric, result in compare.distances().items():
        analysis_results.append({
            'subject': subject,
            'setting': setting,
            'method': row['method'],
            'metric': metric,
            'result': result,
        })

analysis_results = pd.DataFrame(analysis_results)

  0%|          | 0/52 [00:00<?, ?it/s]

In [ ]:
analysis_results